In [37]:
import pandas as pd
import numpy as np
import datetime as dt
from copy import deepcopy
from collections import defaultdict 

In [5]:
rawdata = pd.read_csv("order_brush_order.csv")
print(rawdata.describe())
rawdata.head(10)

            orderid        shopid        userid
count  2.227500e+05  2.227500e+05  2.227500e+05
mean   3.130027e+13  9.433117e+07  9.802880e+07
std    1.222774e+11  5.695790e+07  6.839048e+07
min    3.107520e+13  1.000900e+04  1.000700e+04
25%    3.120360e+13  4.980267e+07  3.508127e+07
50%    3.130561e+13  9.033636e+07  9.309625e+07
75%    3.140604e+13  1.475053e+08  1.590612e+08
max    3.150720e+13  2.154352e+08  2.155262e+08


,orderid,shopid,userid,event_time
0,31076582227611,93950878,30530270,2019-12-27 00:23:03
1,31118059853484,156423439,46057927,2019-12-27 11:54:20
2,31123355095755,173699291,67341739,2019-12-27 13:22:35
3,31122059872723,63674025,149380322,2019-12-27 13:01:00
4,31117075665123,127249066,149493217,2019-12-27 11:37:55
5,31119725718155,173811070,116451780,2019-12-27 12:22:05
6,31151322178251,107921853,166741763,2019-12-27 21:08:43
7,31079856153738,178400128,61272835,2019-12-27 01:17:37
8,31118426867571,147941492,10986763,2019-12-27 12:00:27
9,31133458226149,164933170,135957741,2019-12-27 16:10:59


In [6]:
# rawdata['event_time'] = rawdata['event_time'].apply(lambda x: dt.datetime.strptime(x,"%Y-%m-%d %H:%M:%S"))
rawdata['event_time'] = pd.to_datetime(rawdata['event_time'])
rawdata['shopid'] =rawdata['shopid'].astype(str)
rawdata['userid'] =rawdata['userid'].astype(str)
rawdata.head()


,orderid,shopid,userid,event_time
0,31076582227611,93950878,30530270,2019-12-27 00:23:03
1,31118059853484,156423439,46057927,2019-12-27 11:54:20
2,31123355095755,173699291,67341739,2019-12-27 13:22:35
3,31122059872723,63674025,149380322,2019-12-27 13:01:00
4,31117075665123,127249066,149493217,2019-12-27 11:37:55


In [7]:
# first_order = rawdata.groupby(['shopid'])['event_time'].rank(method='first')
# print(len(first_order))
data_temp = rawdata.sort_values(by=["event_time"], axis=0, ascending=True, na_position ='first')
# first_order = rawdata.groupby('shopid')['event_time'].min()
data_temp

,orderid,shopid,userid,event_time
150060,31075200506751,6042309,97707522,2019-12-27 00:00:00
25985,31075200506752,104804492,97707522,2019-12-27 00:00:00
81020,31075200506753,8715449,97707522,2019-12-27 00:00:00
146598,31075201870570,190969466,170182475,2019-12-27 00:00:02
80355,31075205798264,2859407,12532131,2019-12-27 00:00:05
...,...,...,...,...
114113,31507183252446,149254894,193333760,2019-12-31 23:59:43
206585,31507187390691,147941492,40258063,2019-12-31 23:59:47
126212,31507191066628,187123853,2338306,2019-12-31 23:59:51
160657,31507191066627,154074176,2338306,2019-12-31 23:59:51


In [8]:
tuple_order = list(data_temp.itertuples(index=False, name=None))
tuple_order

[(31075200506751, '6042309', '97707522', Timestamp('2019-12-27 00:00:00')),
 (31075200506752, '104804492', '97707522', Timestamp('2019-12-27 00:00:00')),
 (31075200506753, '8715449', '97707522', Timestamp('2019-12-27 00:00:00')),
 (31075201870570, '190969466', '170182475', Timestamp('2019-12-27 00:00:02')),
 (31075205798264, '2859407', '12532131', Timestamp('2019-12-27 00:00:05')),
 (31075205798265, '94479614', '12532131', Timestamp('2019-12-27 00:00:05')),
 (31075205798263, '65551316', '12532131', Timestamp('2019-12-27 00:00:05')),
 (31075208826809, '147941492', '195903090', Timestamp('2019-12-27 00:00:08')),
 (31075207143538, '54615708', '128755891', Timestamp('2019-12-27 00:00:08')),
 (31075207143537, '127236302', '128755891', Timestamp('2019-12-27 00:00:08')),
 (31075207143539, '137739819', '128755891', Timestamp('2019-12-27 00:00:08')),
 (31075207143536, '26218382', '128755891', Timestamp('2019-12-27 00:00:08')),
 (31075207143540, '18784836', '128755891', Timestamp('2019-12-27 00:

In [10]:
# min_time = min(first_order['event_time'])
# print(min_time)
# max_time = max(first_order['event_time'])
# print(max_time)
# run_time = dt.datetime.strptime(min_time, "%Y-%m-%d %H:%M:%S")
# run_time = run_time + dt.timedelta(hours =1)
# run_time = dt.datetime.strftime(run_time, "%Y-%m-%d %H:%M:%S")
# print(run_time)

In [12]:
# list_shop = list(first_order['shopid'].nunique())
# min_time = min(first_order['event_time'])
# max_time = max(first_order['event_time'])
# list_userid = []
ans = defaultdict(set)
for id, row in enumerate(tuple_order):
    order_time = pd.Timestamp(row[3])
    user_id = row[2]
    end_time = order_time + dt.timedelta(hours =1)
    shop_id = str(row[1])
    if user_id in ans[shop_id]:
        continue
    data_shop = data_temp[id:]
    data_shop = data_shop.loc[(data_shop['shopid']==shop_id) & (data_shop['event_time']>=order_time) & (data_shop['event_time']<=end_time)]
    rate = len(data_shop)/data_shop['userid'].nunique()
    if rate>=3: 
        ans[shop_id].add(user_id)
    
    
    
    
    

In [15]:
len(ans)

18771

In [16]:
ans

defaultdict(set,
            {'6042309': set(),
             '104804492': set(),
             '8715449': {'9753706'},
             '190969466': set(),
             '2859407': set(),
             '94479614': set(),
             '65551316': set(),
             '147941492': set(),
             '54615708': set(),
             '127236302': set(),
             '137739819': set(),
             '26218382': set(),
             '18784836': set(),
             '57934775': set(),
             '31733346': set(),
             '9374147': set(),
             '29975426': set(),
             '316121': set(),
             '141528773': set(),
             '112904482': set(),
             '68862371': {'67554410'},
             '77581273': set(),
             '70519503': set(),
             '64388529': set(),
             '38440816': set(),
             '134869882': set(),
             '72368583': set(),
             '45084333': set(),
             '9466594': set(),
             '5859784': set(),
          

In [34]:


# a = a.groupby('shopid')[]

In [68]:
a = deepcopy(ans)
for a_keys in a.keys():
    if len(a[a_keys])==0:
        a[a_keys] = "0"
    else: 
        user_list = list(a[a_keys])
        user_list.sort(reverse=False)
        a[a_keys] = '&'.join(user_list)

In [69]:
a

defaultdict(set,
            {'6042309': '0',
             '104804492': '0',
             '8715449': '9753706',
             '190969466': '0',
             '2859407': '0',
             '94479614': '0',
             '65551316': '0',
             '147941492': '0',
             '54615708': '0',
             '127236302': '0',
             '137739819': '0',
             '26218382': '0',
             '18784836': '0',
             '57934775': '0',
             '31733346': '0',
             '9374147': '0',
             '29975426': '0',
             '316121': '0',
             '141528773': '0',
             '112904482': '0',
             '68862371': '67554410',
             '77581273': '0',
             '70519503': '0',
             '64388529': '0',
             '38440816': '0',
             '134869882': '0',
             '72368583': '0',
             '45084333': '0',
             '9466594': '0',
             '5859784': '0',
             '96573864': '0',
             '140154637': '0',
         

In [76]:
df =  pd.DataFrame.from_dict(a, orient='index', columns=['userid'])
df = df.reset_index().rename_axis(None, axis=1)
df = df.rename(columns = {"index":"shopid"})
# df.loc[df['shopid']=='181009364',:]
# df = df[:-1]
df.tail()

,shopid,userid
18766,212058195,0
18767,577768,0
18768,162561288,0
18769,203090989,0
18770,0,0


In [77]:
df.to_csv('output_aftersort.csv', header=True,index=None)